In [ ]:
import shutil
import os
import numpy as np
import matplotlib.pylab as plt

import sys
sys.path.append(os.path.abspath("../framework"))
sys.path.append(os.path.abspath("../concrete"))

In [ ]:
from ConcAgent import ConcAgent
from ConcEnvironment import ConcEnvironment
from ConcEnvrionmentFactory import ConcEnvironmentFactory
from ConcAgentFactory import ConcAgentFactory
from ConcAgentMemento import ConcAgentMemento
from ConcBuildOrder import ConcBuildOrder
from ConcStore import ConcStore
from ConcObservation import ConcObservation
from framework import ClosedLoopSimulator
from framework import ObservationSequence 
from framework import TrainId, StoreField
from workUtils import *

## S100. Misc

### SS130. define static variables

In [ ]:
concStore = ConcStore()
agentFactory = ConcAgentFactory()
environmentFactory = ConcEnvironmentFactory()

### SS120. load trainlog into a database

In [ ]:
trainLogFolderPath = "./trainLog"
cur = loadTrainLog(trainLogFolderPath)

# Table TrainLog(
#     buildOrderId Text, /* accord to a build order */
#     trainId   Text, /* accord to a trained agent along with a simulation time*/
#     timeSimulation Integer,
#     timestamp timestamp
# )

## S200. Prepare a simulation

### SS210. choose a build order

In [ ]:
buildOrderIdToPlot = [
    "lMyKekovolegOLpS", # weightOnError = 0.1
    "u0oBOHOk2fPO3bgq", # weightOnError = 0.5
    "JPe2afqJFmAAnowh", # weightOnError = 0.9
    ]
weightOnErrorToPlot = [
    0.1, 
    0.5,
    0.9,
    ]

### SS220. choose the iteration(timesimulation) over the build with the given build order:

In [ ]:
timeSimulationToPlot = [1023, 22527, 44031, 65535]

## SS230. restore the agent and the environment

In [ ]:
def restoreAgentAndEnvironment(buildOrderIdGiven, timeSimulationGiven):
    cur.execute('''
        Select 
            trainId
            From TrainLog
            Where buildOrderId = ?
            And timeSimulation = ?
            ''', (buildOrderIdGiven, timeSimulationGiven))

    trainIdStr, = cur.fetchone()

    # storeField contains: agentMement, buildOrder
    storeField = concStore.load(TrainId(trainIdStr))

    # restore agent
    agent = agentFactory.create(storeField.getBuildOrder())
    agent.loadFromMemento(storeField.getAgentMemento())

    # restrore environment
    environment = environmentFactory.create(storeField.getBuildOrder())
    return agent, environment

## S300. Run a simulation of the given pair of (environment, agent)

In [ ]:
nSeq = 1
nSimu = 2**7

### SS310: run a simulation

In [ ]:
def runSimulation(agent, environment, nSimu):
    Y = [] # observations
    U = [] # actions

    closedLoopSimulator = ClosedLoopSimulator(environment, agent, nSeq)
    closedLoopSimulator.init()
    y = closedLoopSimulator.observationSequenceLast[-1].getValue() # (1, nPv)
    Y.append(y)
    for _ in range(nSimu):
        closedLoopSimulator.update()
        y = closedLoopSimulator.observationSequenceLast[-1].getValue() # (1, nPv)
        u = closedLoopSimulator.actionLast.getActionOnEnvironment() # (1, nMv)
        Y.append(y)
        U.append(u)

    Ynumpy = np.concatenate(Y, axis=0) # (nSimu+1, nPv)
    Unumpy = np.concatenate(U, axis=0) # (nSimu, nMv)
    
    Ty = np.arange(nSimu+1) # (0, 1, ..., 9, 10)
    Tu = np.arange(nSimu) + 0.5 # (0.5, 1.5, 2.5, ..., 9.5)
    
    return Ty, Ynumpy, Tu, Unumpy

### SS320: export the trace of observations and actions to csv

In [ ]:
rows = ["time,value,varType,epoch,weightOnError"]
for buildOrderIdGiven, weightOnErrorGiven in zip(buildOrderIdToPlot, weightOnErrorToPlot):
    for timeSimulationGiven in timeSimulationToPlot:

        agent, environment = restoreAgentAndEnvironment(buildOrderIdGiven, timeSimulationGiven)
        Ty, Ynumpy, Tu, Unumpy = runSimulation(agent, environment, nSimu)

        varType = "Pv"
        for t, y in zip(Ty, Ynumpy):
            row = "{0},{1},{2},iteration={3},weightOnError={4}".format(t, *y, varType, timeSimulationGiven, weightOnErrorGiven)
            rows.append(row)
        
        varType = "Mv"        
        for t, u in zip(Tu, Unumpy):
            row = "{0},{1},{2},iteration={3},weightOnError={4}".format(t, *u, varType, timeSimulationGiven, weightOnErrorGiven)
            rows.append(row)
txt = "\n".join(rows)

with open('data.csv', 'w') as fp:
    fp.write(txt)